In [1]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2
import nltk
import string
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import sys
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

In [3]:
mile3_df = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_train_df_m3.json')

In [4]:
mile3_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48329 entries, 0 to 48328
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bathrooms          48329 non-null  float64
 1   bedrooms           48329 non-null  int64  
 2   building_id        48329 non-null  object 
 3   created            48329 non-null  object 
 4   description        48329 non-null  object 
 5   display_address    48329 non-null  object 
 6   features           48329 non-null  object 
 7   latitude           48329 non-null  float64
 8   listing_id         48329 non-null  int64  
 9   longitude          48329 non-null  float64
 10  manager_id         48329 non-null  object 
 11  photos             48329 non-null  object 
 12  price              48329 non-null  int64  
 13  street_address     48329 non-null  object 
 14  interest_level     48329 non-null  object 
 15  dt_distance        48329 non-null  float64
 16  pet_policy         483

In [13]:
selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "mall_distance", "dt_distance",
                  "pet_policy", "listing_type", "unit_features", "building_features", "apartment",
                  "appliance", "bathroom", "bedroom", "building", "contact", "floor", "high", "kitchen", "room", 
                    "num_photos", "description_len",
                 "managerID_count", "buildingID_count", "weekends", "month_4", "month_5", "created_nights",
                 "created_morning", "created_afternoon", "bed_bath_ratio"]
X = mile3_df[selected_feats]
y = mile3_df["interest_level"]

In [14]:
estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
             ('svr', make_pipeline(StandardScaler(),
                               LinearSVC(random_state=42)))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
clf.fit(X_train, y_train).score(X_test, y_test)

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [9]:
y_val_pred = clf.predict_proba(X_test)
log_loss(y_test, y_val_pred)

0.6050516111078724

In [10]:
df_test = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_test_df_m3.json')
print(df_test.shape)
X = df_test[selected_feats]

y = clf.predict_proba(X)

(74659, 45)


In [11]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}

In [12]:
sub = pd.DataFrame()
sub["listing_id"] = df_test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_newdata_stack.csv", index=False)